<a href="https://colab.research.google.com/github/swati-soulpageit/R/blob/master/Yelp_Nyc_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the data and the libraries required for the same

In [0]:
#libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#libraries
import torch
from torchtext import data

In [0]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True  

In [0]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)

In [0]:
fields = [(None, None), ('text',TEXT),('label', LABEL)]

In [0]:
#loading custom dataset
training_data= data.TabularDataset(path= '/content/drive/My Drive/yelpnyc.csv',format = 'csv',fields = fields,skip_header = True )

#print preprocessed text
print(vars(training_data.examples[0]))

{'text': ['The', 'food', 'at', 'snack', 'is', 'a', 'selection', 'of', 'popular', 'Greek', 'dishes', '.', 'The', 'appetizer', 'tray', 'is', 'good', 'as', 'is', 'the', 'Greek', 'salad', '.', 'We', 'were', 'underwhelmed', 'with', 'the', 'main', 'courses', '.', 'There', 'are', '4', '-', '5', 'tables', 'here', 'so', 'it', "'s", 'sometimes', 'hard', 'to', 'get', 'seated', '.'], 'label': '-1'}


In [0]:
#data cleaning
#def normalise_text (text):
#    text = text.str.lower() # lowercase
#    text = text.str.replace(r"\#","") # replaces hashtags
#    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
#    text = text.str.replace(r"@","")
#    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
#    text = text.str.replace("\s{2,}", " ")
#    return text

In [0]:
#training_data['text'] = normalise_text(training_data['text'])

In [0]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [0]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)   

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                           
100%|█████████▉| 398160/400000 [00:21<00:00, 18965.11it/s]

Size of TEXT vocabulary: 64599
Size of LABEL vocabulary: 2
[('.', 1796110), ('the', 1319027), (',', 1212681), ('and', 969188), ('I', 784620), ('a', 758156), ('to', 597839), ('was', 553751), ('of', 469867), ('\xa0', 424748)]
defaultdict(<function _default_unk_index at 0x7f7c6afe5730>, {'<unk>': 0, '<pad>': 1, '.': 2, 'the': 3, ',': 4, 'and': 5, 'I': 6, 'a': 7, 'to': 8, 'was': 9, 'of': 10, '\xa0': 11, 'it': 12, 'is': 13, 'for': 14, 'in': 15, '!': 16, 'The': 17, 'with': 18, 'but': 19, 'that': 20, 'you': 21, '-': 22, 'on': 23, "n't": 24, "'s": 25, 'my': 26, 'this': 27, 'had': 28, 'were': 29, ')': 30, 'food': 31, 'good': 32, 'place': 33, 'not': 34, 'we': 35, '(': 36, 'have': 37, 'so': 38, 'are': 39, 'they': 40, 'at': 41, 'as': 42, 'be': 43, 'here': 44, 'great': 45, 'like': 46, 'It': 47, 'very': 48, 'just': 49, 'all': 50, '"': 51, 'We': 52, 'there': 53, 'out': 54, 'which': 55, 'really': 56, 'me': 57, '...': 58, 'one': 59, 'get': 60, 'do': 61, 'back': 62, 'would': 63, 'or': 64, 'time': 65, 'g

In [0]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [0]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)
        
        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [0]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout)

In [0]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(64599, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 6,519,357 trainable parameters
torch.Size([64599, 100])


In [0]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def train(model, iterator, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve text and no. of words
        text, text_lengths = batch.text   
        
        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()  
        
        #compute the loss
        loss = criterion(predictions, batch.label)        
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            #retrieve text and no. of words
            text, text_lengths = batch.text
            
            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|█████████▉| 398160/400000 [00:40<00:00, 18965.11it/s]

	Train Loss: 0.309 | Train Acc: 89.71%
	 Val. Loss: 0.296 |  Val. Acc: 89.78%
	Train Loss: 0.280 | Train Acc: 89.79%
	 Val. Loss: 0.307 |  Val. Acc: 89.68%
	Train Loss: 0.246 | Train Acc: 90.54%
	 Val. Loss: 0.318 |  Val. Acc: 88.97%
	Train Loss: 0.204 | Train Acc: 92.04%
	 Val. Loss: 0.380 |  Val. Acc: 88.64%
	Train Loss: 0.162 | Train Acc: 93.67%
	 Val. Loss: 0.439 |  Val. Acc: 86.47%


In [0]:
#load weights
path='/content/saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();

#inference 
import spacy
nlp = spacy.load('en')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence 
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    tensor = tensor.unsqueeze(1).T                             #reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)                  #prediction 
    return prediction.item()          

In [0]:
prediction.

In [34]:
#make predictions
predict(model, "we are in train")

0.299860417842865